In [ ]:
dbutils.fs.unmount("/mnt/mimic")

/mnt/mimic has been unmounted.


True

In [ ]:
# Mounting the Azure Blob Storage to DBFS
dbutils.fs.mount(
  source="wasbs://mimic@mimicsa17.blob.core.windows.net",
  mount_point='/mnt/mimic',
  extra_configs={
    'fs.azure.account.key.mimicsa17.blob.core.windows.net': ""
  }
)

True

In [ ]:
dbutils.fs.ls('/mnt/mimic/raw')

[FileInfo(path='dbfs:/mnt/mimic/raw/admissions.csv', name='admissions.csv', size=47646, modificationTime=1714227575000),
 FileInfo(path='dbfs:/mnt/mimic/raw/d_icd_diagnoses.csv', name='d_icd_diagnoses.csv', size=8959311, modificationTime=1714227587000),
 FileInfo(path='dbfs:/mnt/mimic/raw/drgcodes.csv', name='drgcodes.csv', size=33117, modificationTime=1714227575000),
 FileInfo(path='dbfs:/mnt/mimic/raw/emar.csv', name='emar.csv', size=5271932, modificationTime=1714227582000),
 FileInfo(path='dbfs:/mnt/mimic/raw/patients.csv', name='patients.csv', size=3674, modificationTime=1714227575000),
 FileInfo(path='dbfs:/mnt/mimic/raw/procedures_icd.csv', name='procedures_icd.csv', size=29545, modificationTime=1714227575000),
 FileInfo(path='dbfs:/mnt/mimic/raw/services.csv', name='services.csv', size=14523, modificationTime=1714227575000)]

1.Create a dataframe with proper datatypes

In [ ]:
admissions_schema="subject_id int, hadm_id int, admittime string, dischtime string, deathtime string, admission_type string, admit_provider_id string, admission_location string, discharge_location string, insurance string, language string, marital_status string, race string, edregtime string, edouttime string, hospital_expire_flag int"
patients_schema="subject_id int, gender string, anchor_age int, anchor_year string, anchor_year_group string, dod string"
drgcodes_schema="subject_id int, hadm_id int, drg_type string, drg_code int, description string, drg_severity string, drg_mortality string"
services_schema="subject_id int, hadm_id int, transfertime string, prev_service string, curr_service string"
emar_schema="subject_id int, hadm_id int, emar_id string, emar_seq int, poe_id  string, pharmacy_id int, enter_provider_id string, charttime string, medication string,event_txt string, scheduletime string, storetime string"
d_icd_diagnoses_schema="icd_code int, icd_version int, long_title string"
procedures_icd_schema="subject_id int, hadm_id int, seq_num int, chartdate string, icd_code int, icd_version int"


In [ ]:
df_drgcodes = spark.read.csv("dbfs:/mnt/mimic/raw/drgcodes.csv", schema=drgcodes_schema)
df_admissions = spark.read.csv("dbfs:/mnt/mimic/raw/admissions.csv", schema=admissions_schema)
df_diagnoses = spark.read.csv("dbfs:/mnt/mimic/raw/d_icd_diagnoses.csv", schema=d_icd_diagnoses_schema)
df_emar = spark.read.csv("dbfs:/mnt/mimic/raw/emar.csv", schema=emar_schema)
df_patients = spark.read.csv("dbfs:/mnt/mimic/raw/patients.csv", schema=patients_schema)
df_procedures_icd = spark.read.csv("dbfs:/mnt/mimic/raw/procedures_icd.csv", schema=procedures_icd_schema)
df_services = spark.read.csv("dbfs:/mnt/mimic/raw/services.csv", schema=services_schema)

2. Insert a new column named as ingestion date(current time)


In [ ]:
from pyspark.sql.functions import current_timestamp
df_patients_ingesd = df_patients.withColumn("ingest_date", current_timestamp())
df_drgcode_ingesd = df_drgcodes.withColumn("ingest_date", current_timestamp())
df_admissions_ingesd = df_admissions.withColumn("ingest_date", current_timestamp())
df_diagnoses_ingesd = df_diagnoses.withColumn("ingest_date", current_timestamp())
df_emar_ingesd = df_emar.withColumn("ingest_date", current_timestamp())
df_procedures_ingesd = df_procedures_icd.withColumn("ingest_date", current_timestamp())
df_services_ingesd = df_services.withColumn("ingest_date", current_timestamp())

3.Remove complete duplicate rows

In [ ]:
patients_distinct=df_patients_ingesd.distinct()
drgcode_distinct=df_drgcode_ingesd.distinct()
admissions_distinct=df_admissions_ingesd.distinct()
diagnoses_distinct=df_diagnoses_ingesd.distinct()
emar_distinct=df_emar_ingesd.distinct()
procedures_distinct=df_procedures_ingesd.distinct()
services_distinct=df_services_ingesd.distinct()


4.Remove the null value rows

In [ ]:
patients_distinct_df = patients_distinct.na.drop()
admissions_distinct_df=admissions_distinct.na.drop()

Writing Cleaned data to Cleaned folder in Datalake and into Delta table

In [ ]:
%sql
DROP SCHEMA if EXISTS mimicdb CASCADE;
DROP SCHEMA if EXISTS mimicdbreports CASCADE;

In [ ]:
%sql
create database if not exists mimicdb;
create database if not exists mimicdbreports;

In [ ]:
patients_distinct_df.write.mode("overwrite").format("delta").option("path","/mnt/mimic/cleaned/patients.delta").saveAsTable("mimicdb.patientsdeltatable")
admissions_distinct_df.write.mode("overwrite").format("delta").option("path","/mnt/mimic/cleaned/admissions.delta").saveAsTable("mimicdb.admissionsdeltatable")
drgcode_distinct.write.mode("overwrite").format("delta").option("path","/mnt/mimic/cleaned/drgcode.delta").saveAsTable("mimicdb.drgcodedeltatable")
diagnoses_distinct.write.mode("overwrite").format("delta").option("path","/mnt/mimic/cleaned/diagnoses.delta").saveAsTable("mimicdb.diagnosesdeltatable")
emar_distinct.write.mode("overwrite").format("delta").option("path","/mnt/mimic/cleaned/emar.delta").saveAsTable("mimicdb.emardeltatable")
procedures_distinct.write.mode("overwrite").format("delta").option("path","/mnt/mimic/cleaned/procedures.delta").saveAsTable("mimicdb.proceduresdeltatable")
services_distinct.write.mode("overwrite").format("delta").option("path","/mnt/mimic/cleaned/services.delta").saveAsTable("mimicdb.servicesdeltatable")

In [ ]:
%sql
select * from mimicdb.patientsdeltatable;

subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,ingest_date
10025463,M,66,2136,2011 - 2013,2137-10-09,2024-04-27T21:20:52.867Z
10035631,M,63,2112,2011 - 2013,2116-03-12,2024-04-27T21:20:52.867Z
10032725,F,38,2143,2011 - 2013,2143-03-30,2024-04-27T21:20:52.867Z
10010471,F,89,2155,2014 - 2016,2155-12-07,2024-04-27T21:20:52.867Z
10037861,M,77,2115,2014 - 2016,2117-03-24,2024-04-27T21:20:52.867Z
10021666,M,87,2172,2011 - 2013,2172-04-19,2024-04-27T21:20:52.867Z
10004720,M,61,2183,2014 - 2016,2186-11-17,2024-04-27T21:20:52.867Z
10002930,F,48,2193,2011 - 2013,2201-12-24,2024-04-27T21:20:52.867Z
10015931,M,87,2176,2014 - 2016,2177-03-29,2024-04-27T21:20:52.867Z
10020640,F,91,2153,2011 - 2013,2154-02-04,2024-04-27T21:20:52.867Z


In [ ]:
%sql
describe history mimicdb.patientsdeltatable;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2024-04-27T21:21:15Z,4136898111039167,371efd20-b4d6-4aeb-bfef-b65c112f2ba9,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})","List(532982921670694, ADF_mimicdf_pipeline1_Notebook1_b5136981-b4a0-451f-b2bd-23880ba89c93, null, 400690372824063, 4136898111039167, manual)",List(2425447914677500),0427-211807-peyf1m4t,1,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 31, numOutputBytes -> 2936)",null,Databricks-Runtime/10.4.x-scala2.12
1,2024-04-27T19:01:30Z,760545510687064,vanusakthi@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], description -> null, isManaged -> false, properties -> {}, statsOnLoad -> false)",null,List(2425447914677500),0427-132021-qh391zxk,0,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 31, numOutputBytes -> 2604)",null,Databricks-Runtime/13.3.x-photon-scala2.12
0,2024-04-27T18:33:34Z,4136898111039167,371efd20-b4d6-4aeb-bfef-b65c112f2ba9,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})","List(318101451921577, ADF_mimicdf_pipeline1_Notebook1_dda02c74-2ca6-4360-82ce-5c511b167a52, null, 415008752895433, 4136898111039167, manual)",List(2425447914677500),0427-183056-hemee69g,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 31, numOutputBytes -> 2936)",null,Databricks-Runtime/10.4.x-scala2.12


In [ ]:
%sql
INSERT INTO mimicdb.patientsdeltatable VALUES (10024045, 'M', 99, '2170', '2011-2013', '2134-10-28', '2024-04-27T21:20:52.867+00:00')

num_affected_rows,num_inserted_rows
1,1


In [ ]:
%sql
describe history mimicdb.patientsdeltatable;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2024-04-27T22:18:42Z,760545510687064,vanusakthi@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2425447914677500),0427-132021-qh391zxk,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1767)",null,Databricks-Runtime/13.3.x-photon-scala2.12
2,2024-04-27T21:21:15Z,4136898111039167,371efd20-b4d6-4aeb-bfef-b65c112f2ba9,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})","List(532982921670694, ADF_mimicdf_pipeline1_Notebook1_b5136981-b4a0-451f-b2bd-23880ba89c93, null, 400690372824063, 4136898111039167, manual)",List(2425447914677500),0427-211807-peyf1m4t,1,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 31, numOutputBytes -> 2936)",null,Databricks-Runtime/10.4.x-scala2.12
1,2024-04-27T19:01:30Z,760545510687064,vanusakthi@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], description -> null, isManaged -> false, properties -> {}, statsOnLoad -> false)",null,List(2425447914677500),0427-132021-qh391zxk,0,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 31, numOutputBytes -> 2604)",null,Databricks-Runtime/13.3.x-photon-scala2.12
0,2024-04-27T18:33:34Z,4136898111039167,371efd20-b4d6-4aeb-bfef-b65c112f2ba9,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})","List(318101451921577, ADF_mimicdf_pipeline1_Notebook1_dda02c74-2ca6-4360-82ce-5c511b167a52, null, 415008752895433, 4136898111039167, manual)",List(2425447914677500),0427-183056-hemee69g,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 31, numOutputBytes -> 2936)",null,Databricks-Runtime/10.4.x-scala2.12


In [ ]:
%sql
select * from mimicdb.patientsdeltatable where subject_id=10031757;

subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,ingest_date
10031757,F,67,2137,2014 - 2016,2137-10-31,2024-04-27T21:20:52.867Z


In [ ]:
%sql
update mimicdb.patientsdeltatable set gender ='M' where subject_id=10031757;

num_affected_rows
1


In [ ]:
%sql
select * from mimicdb.patientsdeltatable where subject_id=10031757;

subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,ingest_date
10031757,M,67,2137,2014 - 2016,2137-10-31,2024-04-27T21:20:52.867Z


In [ ]:
%sql
describe history mimicdb.patientsdeltatable;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2024-04-27T22:29:45Z,760545510687064,vanusakthi@gmail.com,UPDATE,"Map(predicate -> [""(subject_id#2452 = 10031757)""])",null,List(2425447914677500),0427-132021-qh391zxk,3,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 2936, numCopiedRows -> 30, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1857, scanTimeMs -> 1119, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 2616, rewriteTimeMs -> 726)",null,Databricks-Runtime/13.3.x-photon-scala2.12
3,2024-04-27T22:18:42Z,760545510687064,vanusakthi@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2425447914677500),0427-132021-qh391zxk,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1767)",null,Databricks-Runtime/13.3.x-photon-scala2.12
2,2024-04-27T21:21:15Z,4136898111039167,371efd20-b4d6-4aeb-bfef-b65c112f2ba9,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})","List(532982921670694, ADF_mimicdf_pipeline1_Notebook1_b5136981-b4a0-451f-b2bd-23880ba89c93, null, 400690372824063, 4136898111039167, manual)",List(2425447914677500),0427-211807-peyf1m4t,1,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 31, numOutputBytes -> 2936)",null,Databricks-Runtime/10.4.x-scala2.12
1,2024-04-27T19:01:30Z,760545510687064,vanusakthi@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], description -> null, isManaged -> false, properties -> {}, statsOnLoad -> false)",null,List(2425447914677500),0427-132021-qh391zxk,0,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 31, numOutputBytes -> 2604)",null,Databricks-Runtime/13.3.x-photon-scala2.12
0,2024-04-27T18:33:34Z,4136898111039167,371efd20-b4d6-4aeb-bfef-b65c112f2ba9,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})","List(318101451921577, ADF_mimicdf_pipeline1_Notebook1_dda02c74-2ca6-4360-82ce-5c511b167a52, null, 415008752895433, 4136898111039167, manual)",List(2425447914677500),0427-183056-hemee69g,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 31, numOutputBytes -> 2936)",null,Databricks-Runtime/10.4.x-scala2.12


In [ ]:
%sql
delete from mimicdb.patientsdeltatable where subject_id=10024045

num_affected_rows
1


Joining Admissions and Patients dataframe for optimisation 

In [ ]:
merged_df = admissions_distinct.join(patients_distinct, "subject_id")

In [ ]:
from pyspark.sql.functions import countDistinct

# Usecase1
gender_admitted_df = merged_df.groupBy("gender").agg(countDistinct("subject_id").alias("count_subjects"))
# Repartition the DataFrame before writing
gender_admitted_df = gender_admitted_df.coalesce(1)
gender_admitted_df.show()
# Write the DataFrame to Delta Lake
gender_admitted_df.write.mode("overwrite").format("delta").option("path","/mnt/mimic/processed/Gender_admissions.delta").saveAsTable("mimicdbreports.usecase1")


+------+--------------+
|gender|count_subjects|
+------+--------------+
|     F|            43|
|     M|            57|
+------+--------------+



In [ ]:
from pyspark.sql.functions import countDistinct

#Usecase2 Male death vs Female death
gender_death_df =  merged_df \
        .where(df_admissions["hospital_expire_flag"] == 1) \
        .groupBy("gender").agg(countDistinct("subject_id").alias("count_subjects"))
# Repartition the DataFrame before writing
gender_death_df = gender_death_df.coalesce(1)
gender_death_df.show()
# Write the DataFrame to Delta Lake
gender_death_df.write.mode("overwrite").format("delta").option("path","/mnt/mimic/processed/Gender_death.delta").saveAsTable("mimicdbreports.usecase2")

+------+--------------+
|gender|count_subjects|
+------+--------------+
|     F|             3|
|     M|            12|
+------+--------------+



In [ ]:
#Usecase3 Admission location Discharge location stats
discharge_location_df =  merged_df \
        .groupBy("discharge_location").agg(countDistinct("hadm_id").alias("count_hadm_id"))
admission_discharge_df =  merged_df \
        .groupBy("admission_location", "discharge_location") \
        .agg(countDistinct("hadm_id").alias("count_hadm_id"))
# Repartition the DataFrame before writing
admission_discharge_df = admission_discharge_df.coalesce(1)
admission_discharge_df.show()
# Write the DataFrame to Delta Lake
admission_discharge_df.write.mode("overwrite").format("delta").option("path","/mnt/mimic/processed/Discharge_location.delta").saveAsTable("mimicdbreports.usecase3")

+--------------------+--------------------+-------------+
|  admission_location|  discharge_location|count_hadm_id|
+--------------------+--------------------+-------------+
|TRANSFER FROM HOS...|CHRONIC/LONG TERM...|            1|
|                PACU|                HOME|            1|
|      EMERGENCY ROOM|             HOSPICE|            3|
|      EMERGENCY ROOM|      AGAINST ADVICE|            2|
|      PROCEDURE SITE|               REHAB|            1|
|      PROCEDURE SITE|                NULL|            3|
|      EMERGENCY ROOM|    HOME HEALTH CARE|           32|
|  PHYSICIAN REFERRAL|    HOME HEALTH CARE|           21|
|      EMERGENCY ROOM|                NULL|           18|
|     CLINIC REFERRAL|                HOME|            3|
|TRANSFER FROM HOS...|               REHAB|            2|
|      EMERGENCY ROOM|      PSYCH FACILITY|            2|
|TRANSFER FROM HOS...|      ACUTE HOSPITAL|            1|
|INTERNAL TRANSFER...|                HOME|            2|
|  PHYSICIAN R

In [ ]:
# Usecase4 Age Group admissions
from pyspark.sql.functions import when, col
age_group_df = merged_df \
.select("subject_id","anchor_age").distinct() \
    .withColumn("age_group", when((col("anchor_age") >= 1) & (col("anchor_age") <= 10), "0-10")
                .when((col("anchor_age") >= 11) & (col("anchor_age") <= 20), "11-20")
                .when((col("anchor_age") >= 21) & (col("anchor_age") <= 30), "21-30")
                .when((col("anchor_age") >= 31) & (col("anchor_age") <= 40), "31-40")
                .when((col("anchor_age") >= 41) & (col("anchor_age") <= 50), "41-50")
                .when((col("anchor_age") >= 51) & (col("anchor_age") <= 60), "51-60")
                .when((col("anchor_age") >= 61) & (col("anchor_age") <= 70), "61-70")
                .when((col("anchor_age") >= 71) & (col("anchor_age") <= 80), "71-80")
                .when((col("anchor_age") >= 81) & (col("anchor_age") <= 90), "81-90")
                .when((col("anchor_age") >= 91) & (col("anchor_age") <= 100), "91-100")
                .otherwise("null"))
age_group_counts = age_group_df.groupBy("age_group").count().orderBy("age_group")
# Repartition the DataFrame before writing
age_group_counts = age_group_counts.coalesce(1)
age_group_counts.show()
# Write the DataFrame to Delta Lake
age_group_counts.write.mode("overwrite").format("delta").option("path","/mnt/mimic/processed/Age_group.delta").saveAsTable("mimicdbreports.usecase4")

+---------+-----+
|age_group|count|
+---------+-----+
|    21-30|    5|
|    31-40|    5|
|    41-50|   13|
|    51-60|   22|
|    61-70|   28|
|    71-80|   12|
|    81-90|   12|
|   91-100|    3|
+---------+-----+



In [ ]:
#Usecase5 No of patients got discharged and not admitted again = No Readmissions
filtered_df = merged_df.filter(merged_df["discharge_location"].isNull())
result_df = filtered_df.select("subject_id").agg(countDistinct("subject_id").alias("Patients_discharged_without_readmission"))
# Repartition the DataFrame before writing
result_df = result_df.coalesce(1)
result_df.show()
# Write the DataFrame to Delta Lake
result_df.write.mode("overwrite").format("delta").option("path","/mnt/mimic/processed/Discharged_no_readmission.delta").saveAsTable("mimicdbreports.usecase5")

+---------------------------------------+
|Patients_discharged_without_readmission|
+---------------------------------------+
|                                     18|
+---------------------------------------+



In [ ]:
#Usecase6 No of patients admitted under each category
admission_types = merged_df.groupBy("admission_type").agg(countDistinct("subject_id").alias("admissions"))
# Repartition the DataFrame before writing
admission_types = admission_types.coalesce(1)
admission_types.show()
# Write the DataFrame to Delta Lake
admission_types.write.mode("overwrite").format("delta").option("path","/mnt/mimic/processed/Category_type.delta").saveAsTable("mimicdbreports.usecase6")

+--------------------+----------+
|      admission_type|admissions|
+--------------------+----------+
|  DIRECT OBSERVATION|         6|
|      EU OBSERVATION|        13|
|            EW EMER.|        60|
|            ELECTIVE|        10|
|SURGICAL SAME DAY...|        17|
|   OBSERVATION ADMIT|        22|
|AMBULATORY OBSERV...|         4|
|              URGENT|        32|
|        DIRECT EMER.|        12|
+--------------------+----------+



In [ ]:
#Usecase7 Insurance category
insurance_category_df = merged_df.groupBy("insurance").agg(countDistinct("subject_id").alias("count_subjects"))
# Repartition the DataFrame before writing
insurance_category_df = insurance_category_df.coalesce(1)
insurance_category_df.show()
# Write the DataFrame to Delta Lake
insurance_category_df.write.mode("overwrite").format("delta").option("path","/mnt/mimic/processed/Insurance_category.delta").saveAsTable("mimicdbreports.usecase7")

+---------+--------------+
|insurance|count_subjects|
+---------+--------------+
|    Other|            57|
| Medicaid|            10|
| Medicare|            38|
+---------+--------------+



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, countDistinct

spark = SparkSession.builder.getOrCreate()

# Usecase8 Admission count in each department
joined_df = merged_df.join(services_distinct, ['subject_id', 'hadm_id'], 'left')
result_df = joined_df.groupBy('curr_service') \
    .agg(countDistinct('hadm_id').alias('admission_count')) \
    .orderBy(desc('admission_count'))

# Repartition the DataFrame before writing
admission_types = admission_types.coalesce(1)
admission_types.show()
# Write the DataFrame to Delta Lake
admission_types.write.mode("overwrite").format("delta").option("path","/mnt/mimic/processed/Admission_type.delta").saveAsTable("mimicdbreports.usecase8")

+--------------------+----------+
|      admission_type|admissions|
+--------------------+----------+
|  DIRECT OBSERVATION|         6|
|      EU OBSERVATION|        13|
|            EW EMER.|        60|
|            ELECTIVE|        10|
|SURGICAL SAME DAY...|        17|
|   OBSERVATION ADMIT|        22|
|AMBULATORY OBSERV...|         4|
|              URGENT|        32|
|        DIRECT EMER.|        12|
+--------------------+----------+



In [ ]:
#Usecase9 #Usecase9 Medication prescribed for each diagnosis
from pyspark.sql.functions import countDistinct, array_join, collect_list, col
joined_df = merged_df.join(emar_distinct,(emar_distinct.subject_id==merged_df.subject_id) 
                           & (emar_distinct.hadm_id==merged_df.hadm_id), 'left')
joined1_df = joined_df.join(procedures_distinct,(procedures_distinct.subject_id==emar_distinct.subject_id) 
                            & (procedures_distinct.hadm_id==emar_distinct.hadm_id), 'left')
joined2_df = joined1_df.join(diagnoses_distinct, (diagnoses_distinct.icd_code == joined1_df.icd_code) 
                             & (diagnoses_distinct.icd_version == joined1_df.icd_version), 'left')
filtered_df = joined2_df.filter((col('long_title').isNotNull()) & (col('medication').isNotNull()))
filtered1_df=filtered_df.withColumnRenamed('long_title','Diagnosis')
result_df = filtered1_df.groupBy('Diagnosis') \
    .agg(countDistinct('medication').alias('Medication_count'),array_join(collect_list('medication'), ', ').alias('Medication_list')) \
    .orderBy('Diagnosis')
result_df.show()
# Write the DataFrame to Delta Lake
result_df.write.mode("overwrite").format("delta").option("path","/mnt/mimic/processed/Medication_prescribed.delta").saveAsTable("mimicdbreports.usecase9")

+--------------------+----------------+--------------------+
|           Diagnosis|Medication_count|     Medication_list|
+--------------------+----------------+--------------------+
|Acute nonparalyti...|              53|Acetaminophen, Ac...|
|Acute poliomyelit...|              67|Acetaminophen, Ac...|
|Alcohol-induced p...|              29|Acetaminophen, Ac...|
|Benign essential ...|              11|Acetaminophen, Ac...|
|Benign neoplasm o...|              29|Acetaminophen, Ac...|
|Benign neoplasm o...|              25|Albumin 25% (12.5...|
|Benign neoplasm o...|              25|Albumin 25% (12.5...|
|Blisters, epiderm...|              14|Acetaminophen, Ac...|
|          Chilblains|              39|Acetaminophen, Ac...|
|Contusion of unsp...|               2|Morphine Sulfate,...|
|Deaf, nonspeaking...|              93|Acetaminophen, Ac...|
|Diaphragmatic her...|              26|Alteplase 1mg/2mL...|
|   Frostbite of face|               4|Morphine Sulfate,...|
|Glossopharyngeal ...|  

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import col, count

# Usecase 10: Patients admitted with Severity and Mortality High who have Recovered
joined_df = merged_df.join(drgcode_distinct, ["subject_id", "hadm_id"], 'left')
filtered_df = joined_df.filter((col("drg_severity") > 3) & (col("drg_mortality") > 3))
window_spec = Window.partitionBy("hospital_expire_flag").orderBy("hospital_expire_flag")
result_df = filtered_df.select("hospital_expire_flag", count("hadm_id").over(window_spec).alias("count"))
distinct_result_df = result_df.dropDuplicates(["hospital_expire_flag"])
recovered_df = distinct_result_df.filter(col("hospital_expire_flag") == 0)
recovered_df = recovered_df.withColumnRenamed("hospital_expire_flag", "Recovered")
# Repartition the DataFrame before writing
recovered_df = admission_types.coalesce(1)
recovered_df.show()
# Write the DataFrame to Delta Lake
recovered_df.write.mode("overwrite").format("delta").option("path","/mnt/mimic/processed/Recovered_count.delta").saveAsTable("mimicdbreports.usecase10")

+--------------------+----------+
|      admission_type|admissions|
+--------------------+----------+
|  DIRECT OBSERVATION|         6|
|      EU OBSERVATION|        13|
|            EW EMER.|        60|
|            ELECTIVE|        10|
|SURGICAL SAME DAY...|        17|
|   OBSERVATION ADMIT|        22|
|AMBULATORY OBSERV...|         4|
|              URGENT|        32|
|        DIRECT EMER.|        12|
+--------------------+----------+

